# JSON & SNS Crawling

## JSON이란?
 - 자바스크립트(JavaScript): 웹브라우저에서 작동하는 프로그래밍 언어
 - JSON(JavaScript Object Notation): 자바스크립트의 사전(dictionary) 형식
 - 웹브라우저와 서버 사이에 간단히 자료를 주고 받을 때 사용
 - '더 보기' 형태로 웹페이지를 이동하지 않고 내용이 추가되는 경우 대부분 서버에서 JSON 형식으로 자료를 받아 자바스크립트가 화면에 추가
 - 따라서 JSON 자료를 가로채면 쉽게 크롤링을 할 수 있음

## 예시 

In [1]:
sample = '''
{
   "id": 123,
   "username": "유재명",
   "comment": "반갑습니다"
}
'''

In [2]:
sample

'\n{\n   "id": 123,\n   "username": "유재명",\n   "comment": "반갑습니다"\n}\n'

## JSON 해석

In [3]:
import json

In [46]:
from urllib.parse import quote_plus

In [4]:
json.loads(sample)

{'comment': '반갑습니다', 'id': 123, 'username': '유재명'}

## 네이버 뉴스 댓글 JSON으로 가져오기

In [5]:
origin_url = 'http://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=105&oid=079&aid=0002971594&m_view=1'

In [131]:
origin_url_tmp = 'http://sports.news.naver.com/wfootball/news/read.nhn?oid=139&aid=0002075732&m_view=1&sort=LIKE'

 - CallBack은 대충 그냥 맞추면 되는거같다. 

In [149]:
ticket = 'news'

In [150]:
news_id = '079'

In [151]:
news = "news" + news_id

In [152]:
aid = '0002971594'

In [159]:
objectId =  news + "," + aid

In [160]:
objectId = quote_plus(objectId)
objectId

'news079%2C0002971594'

In [161]:
page = 1 

In [162]:
comment_url = ('https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?'
               'ticket=news&templateId=default_it&pool=cbox5&_callback=jQuery1709951719079191432_1496387548832&'
               'lang=ko&country=KR&objectId=news079%2C0002971594&categoryId=&pageSize=20&indexSize=10&groupId=&'
               'listType=OBJECT&page=1&initialize=true&userType=&useAltSort=true&replyPageSize=20&moveTo=&'
               'sort=reply&_=1496387549604')

In [163]:
comment_url = ('https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?'
               'ticket={ticket}&templateId=default_it&pool=cbox5&_callback=jQuery1709951719079191433_1496387548832&'
               'lang=ko&country=KR&objectId={objectId}&categoryId=&pageSize=20&indexSize=10&groupId=&'
               'listType=OBJECT&page={page}&initialize=true&userType=&useAltSort=true&replyPageSize=20&moveTo=&'
               'sort=reply&_=1496387549604')

In [164]:
tmp_url = comment_url.format(ticket=ticket, objectId=objectId,page=page)

In [165]:
tmp_url

'https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_it&pool=cbox5&_callback=jQuery1709951719079191433_1496387548832&lang=ko&country=KR&objectId=news079%2C0002971594&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&page=1&initialize=true&userType=&useAltSort=true&replyPageSize=20&moveTo=&sort=reply&_=1496387549604'

In [166]:
import requests

In [167]:
res = requests.get(tmp_url, headers = {'referer':origin_url_tmp})

In [168]:
res.status_code

200

In [169]:
res.text[:100]

'jQuery1709951719079191433_1496387548832({"success":true,"code":"1000","message":"요청을 성공적으로 처리하였습니다."'

In [171]:
res.text[:500]

'jQuery1709951719079191433_1496387548832({"success":true,"code":"1000","message":"요청을 성공적으로 처리하였습니다.","lang":"ko","country":"KR","result":{"sort":"REPLY","count":{"comment":58,"reply":32,"exposeCount":59,"delCommentByUser":4,"total":90},"config":{"useSnsLogin":true,"lineFeedOn":false,"useBest":false,"useVote":true,"useVoteSelf":false,"useVoteGoodOnly":false,"useReport":true,"useCommonReport":true,"useSort":true,"sortTypes":["FAVORITE","NEW","OLD","REPLY"],"defaultSort":"FAVORITE","useByteLength":'

In [172]:
json.loads(res.text)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [75]:
start = len('jQuery1709951719079191432_1496387548832(')
end = len(');')

In [76]:
comment = json.loads(res.text[start:-end])

In [77]:
len(comment)

7

In [79]:
comment['result']['commentList']

[]

In [27]:
comment['result']['commentList'][0]['contents']

'수학 난제 증명에도 사용가능하게 된다면 인간에겐 또한번의 불의 발견 같은 사건이 될것이다.'

# 인스타그램에서 JSON으로 댓글 가져오기

In [173]:
import lxml.html

In [174]:
url = 'https://www.instagram.com/p/BUCPaLQBb2r/?taken-by=dlwlrma&hl=ko'

In [175]:
res = requests.get(url)

In [176]:
res.status_code

200

In [177]:
root = lxml.html.fromstring(res.text)

In [178]:
scripts = root.cssselect('script')

In [179]:
scripts[3].text

'window._sharedData = {"activity_counts": null, "config": {"csrf_token": "PDTYmLm6oRt8wXbkbT69SGw7tSrK3zos", "viewer": null}, "country_code": "KR", "language_code": "ko", "entry_data": {"PostPage": [{"graphql": {"shortcode_media": {"__typename": "GraphImage", "id": "1513840194233941419", "shortcode": "BUCPaLQBb2r", "dimensions": {"height": 1349, "width": 1080}, "display_url": "https://scontent-hkg3-2.cdninstagram.com/t51.2885-15/e35/18444702_512530218870867_2575925065480667136_n.jpg", "is_video": false, "edge_media_to_tagged_user": {"edges": []}, "edge_media_to_caption": {"edges": [{"node": {"text": "\\uc9c0\\ub098\\uac00\\ub2e4 #\\ube60\\ub808\\ud2b8 \\ubc1c\\uacac!"}}]}, "caption_is_edited": false, "edge_media_to_comment": {"count": 3418, "page_info": {"has_next_page": true, "end_cursor": "AQBOio5HrVwpHf77C6PvNIVqW0-G-s1IF_A4KVamks8jpo2rzInBQZGaldrc_VbpH5Zso9Tfo5HCiaGu4ckwI4xlF5-XAWpVm1sutkuv4M3Y7w"}, "edges": [{"node": {"id": "17870442001117880", "text": "4M\\ud83d\\udc4f\\ud83c\\ud

In [180]:
start = len('window._sharedData = ')
end = len(';')

In [182]:
comments = json.loads(scripts[3].text[start:-end])

In [198]:
comments

{'activity_counts': None,
 'config': {'csrf_token': 'PDTYmLm6oRt8wXbkbT69SGw7tSrK3zos', 'viewer': None},
 'country_code': 'KR',
 'display_properties_server_guess': {'pixel_ratio': 1.5,
  'viewport_width': 360},
 'entry_data': {'PostPage': [{'graphql': {'shortcode_media': {'__typename': 'GraphImage',
      'caption_is_edited': False,
      'comments_disabled': False,
      'dimensions': {'height': 1349, 'width': 1080},
      'display_url': 'https://scontent-hkg3-2.cdninstagram.com/t51.2885-15/e35/18444702_512530218870867_2575925065480667136_n.jpg',
      'edge_media_preview_like': {'count': 449763,
       'edges': [{'node': {'id': '2083088509',
          'profile_pic_url': 'https://scontent-hkg3-2.cdninstagram.com/t51.2885-19/s150x150/18644738_213634639155044_5154215902027186176_a.jpg',
          'username': '14l_arruda'}},
        {'node': {'id': '3678325338',
          'profile_pic_url': 'https://scontent-hkg3-2.cdninstagram.com/t51.2885-19/s150x150/18808813_113381642583812_3652875955

In [188]:
def find_key(dic, key):
    path = []
    if isinstance(dic, list) and dic:
        result = find_key(dic[0], key)
        if result:
             path = [0] + result
        
    elif isinstance(dic, dict):
        if key in dic:
            path = [key]
        else:
            for k in dic:
                result = find_key(dic[k], key)
                if result:
                    path = [k] + result
    #print(path)
    return path

In [189]:
find_key(comments, 'edge_media_to_comment')

['entry_data',
 'PostPage',
 0,
 'graphql',
 'shortcode_media',
 'edge_media_to_comment']

In [190]:
keys = find_key(comments, 'edge_media_to_comment')

In [192]:
print(''.join("[{}]".format(repr(k)) for k in keys)) 

['entry_data']['PostPage'][0]['graphql']['shortcode_media']['edge_media_to_comment']


In [193]:
comments['entry_data']['PostPage'][0]['graphql']['shortcode_media']['edge_media_to_comment']

{'count': 3418,
 'edges': [{'node': {'created_at': 1496068518,
    'id': '17870442001117880',
    'owner': {'id': '1327739384',
     'profile_pic_url': 'https://scontent-hkg3-2.cdninstagram.com/t51.2885-19/s150x150/15803573_1353229571364124_8017508643643588608_n.jpg',
     'username': 'aiyx3'},
    'text': '4M👏🏻👏🏻👏🏻❤💋'}},
  {'node': {'created_at': 1496072119,
    'id': '17860120810140312',
    'owner': {'id': '2175864049',
     'profile_pic_url': 'https://scontent-hkg3-2.cdninstagram.com/t51.2885-19/s150x150/16122957_1723722997653429_960675847860125696_a.jpg',
     'username': 'shjiong'},
    'text': '💜💜💜💜💜💜'}},
  {'node': {'created_at': 1496072334,
    'id': '17882149579039206',
    'owner': {'id': '2250297403',
     'profile_pic_url': 'https://scontent-hkg3-2.cdninstagram.com/t51.2885-19/s150x150/18161849_109889446245744_6129760827881816064_a.jpg',
     'username': 'sjji__'},
    'text': '내츄럴함이 가장 매력인 지은님\U0001f921❣️'}},
  {'node': {'created_at': 1496074550,
    'id': '17873191846077

In [215]:
a =  comments['entry_data']['PostPage'][0]['graphql']['shortcode_media']['edge_media_to_comment'] 
for i in range(len(a['edges'])):
    print(a['edges'][i]['node']['text'])

4M👏🏻👏🏻👏🏻❤💋
💜💜💜💜💜💜
내츄럴함이 가장 매력인 지은님🤡❣️
Nice!💓💓
바보야😈
시발👹
나쁜아주마😈😬👿
Закрой рот @sweeetie_01
@hoz_0427 дура за к
Кого ты заступаешся @hoz_0427
😭
Bebek gibi görünmeye çalışma maymun
Just i like u so much💝pink😘
IU I love you so much and I'm glad that you're looking good and happy after your trip in jeju. Do not mind those comments and I'm sure you are only taking them as your milk after feeding on us star candies 😀❤❤❤
Saranghae~ IU sunbaenim ♥ ^^
If you hate her that much why don't you just STFU!!! And don't make her sad!!!! If anything happens bad to IU I'm gonna fucking kill that person who makes IU sad!
님 소듀 좋아해요?
Weh kan. Ana buu tp angel namane wkwkwk @rizaikhoirunnisa
언니 이뻐요 💞💞💞💕💕
😊
예쁜 아이유언니🙂🙂🙂
long time no see…🤓
언니 사랑해용 💞 💞 💞
🌼🌼🌼🌼
内心煎熬，而我却说不出口
팔레트 노래 진짜 너무 너무 좋아요. 이렇게 좋은 음악 들려주셔서 정말 감사해요. 지은님 노래중에 제일 명곡인 것 같아요.모든 뮤지션중에서 젤루 사랑합니다.
Gorg💕
cute
💕💕💕 doluşmuş yine hatercılar yerim ulan 💕💕💕💕💕
maymuşum benim 💕💕💕💕
💝💝💝💝💝💝 my monkey, i love u so much.
I was really shocked when i saw suho on the f

In [194]:
find_key(comments, 'end_cursor')

['entry_data',
 'PostPage',
 0,
 'graphql',
 'shortcode_media',
 'edge_media_to_comment',
 'page_info',
 'end_cursor']

In [195]:
keys = find_key(comments, 'end_cursor')
print(''.join("[{}]".format(repr(k)) for k in keys)) 

['entry_data']['PostPage'][0]['graphql']['shortcode_media']['edge_media_to_comment']['page_info']['end_cursor']


In [197]:
comments['entry_data']['PostPage'][0]['graphql']['shortcode_media']['edge_media_to_comment']['page_info']['end_cursor']

'AQBOio5HrVwpHf77C6PvNIVqW0-G-s1IF_A4KVamks8jpo2rzInBQZGaldrc_VbpH5Zso9Tfo5HCiaGu4ckwI4xlF5-XAWpVm1sutkuv4M3Y7w'

## 댓글 더보기 

In [216]:
res = requests.get('https://www.instagram.com/graphql/query/',
                   params={'query_id': '17852405266163336',
                     'shortcode': 'BUCPaLQBb2r',
                     'first': 21,
                     'after': comments['entry_data']['PostPage'][0]['graphql']['shortcode_media']['edge_media_to_comment']['page_info']['end_cursor']
                    })

In [217]:
res.json()

{'data': {'shortcode_media': {'edge_media_to_comment': {'count': 3418,
    'edges': [{'node': {'created_at': 1495947070,
       'id': '17870432371103515',
       'owner': {'id': '5098231234',
        'profile_pic_url': 'https://scontent-hkg3-2.cdninstagram.com/t51.2885-19/s150x150/17881062_156327741559903_2590241690421821440_a.jpg',
        'username': 'liukaka1'},
       'text': '小仙女'}},
     {'node': {'created_at': 1495954044,
       'id': '17857906363158409',
       'owner': {'id': '3523218395',
        'profile_pic_url': 'https://scontent-hkg3-2.cdninstagram.com/t51.2885-19/s150x150/13725706_474103149447019_415180542_a.jpg',
        'username': 'abc05160918'},
       'text': '빠레트 다음에 근처가면 앞에서 사진찍어야지'}},
     {'node': {'created_at': 1495963146,
       'id': '17869336522121849',
       'owner': {'id': '5517226459',
        'profile_pic_url': 'https://scontent-hkg3-2.cdninstagram.com/t51.2885-19/s150x150/18723782_303427606781600_4925868987868250112_a.jpg',
        'username': 'iuiu199

## 트위터 

In [226]:
res = requests.get('https://twitter.com/i/search/timeline?vertical=default&q=고양이')

In [227]:
result = res.json()

In [228]:
result.keys()

dict_keys(['new_latent_count', 'focused_refresh_interval', 'items_html', 'has_more_items'])

In [231]:
with open('twitter.html','w',encoding='utf-8') as f:
    f.write(result['items_html'])

In [232]:
root = lxml.html.fromstring(result['items_html'])

In [234]:
for tweet in root.cssselect('.js-tweet-text-container'):
    print(tweet.text_content().split('pic')[0])


  선생님 제 고양이가 너무 토끼같이 예뻐요 

  신을 집사로 둔 고양이3 

  긱숙사에 고양이 들어왔어요 여러분 이것봐 너무 귀여워ㅠㅠㅜㅠㅜㅜㅜㅜㅜㅠ 

  이렇게까지 말하는 내가
이렇게까지 필요없나요
차갑게 식은 그대란 건 알지만
난 그대가 필요해요

  고양이가 지나간 자리,.. 

  170602 소매 쥔 주먹도 웃는 것두 완전 아기 고양이 아니냥 

  어서 밤고양이가 되어줘~~~! 

  대형산불 막으려 물배낭 메고 직접 수락산 오른 외국인http://www.insight.co.kr/newsRead.php?ArtNo=107910 …


  테루테루 이치카라 보고 있으면 그거.. 아아 훌륭한 고양이의 삶 가사 생각난다
귀여운 아가씨 새하얀 털이 굉장히 멋지군요 

  용국이가 너무 귀여워서 움짤까지 만들었다,, 그저 고양이 인간 ㅜㅡㅜ

https://i.imgur.com/Zr1j50K.gifv 

#김용국 #열어줘

  <===린유즈님이 생각했을 킨조
우리가 생각하는 킨조====>

  택연이 형아 가슴팍을 팡!팡!팡! #우영 #택연

  다리가 부러져도 귀여운게 고양이.... 어흐흑 하트무너진다 

  이것은 기분이 좋으면 앞발로 젖을 빠는 고양이이다. 

  카즈키고양이:(상처받은표정)


  인간만 그런게 아니다. 솔로 냥이들도 커플 냥이들 보면 눈꼴 시려워한다냥....

  나 옹이 저렇게 윙 따라하는 거 보고 생각난건데 나중에 머뷔한 다음에 스피드퀴즈 같은 거에서 답이 아기고양이 이런 거라 옹이 "작고 귀여운 거! 야옹야옹!" 이러니까 녤이 존나 손 들면서 다급하게 "박지훉!!" 이래서 윙 존나 황당해하는 거 보고 싶다


  너 고양이야? 

  아기고양이 세 마리가 태어난 걸 확인하니 어깨가 무거워졌다,고 알바님께 말했더니... 이런 그림을 그려와 메뉴판에 붙여주심. ㅎㅎㅎ 

  RT추첨) 
뉴 고양이 선풍기 판매합니다
 인증받은 안전한 선풍기
 바람 3단 조절 가능! 민트품절
택포 11000원 / 입금 다음날 배송
[뉴고양이선풍